Bilan: assez mauvais résultats, ne prévoit pas bien si ce sera une hausse ou une baisse

In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [20]:
csv_path = 'MC.PA.csv'
date = '2023-04-13'

data = pd.read_csv(csv_path)

data_open = data[['Date', 'Open']]
data_open = data_open.fillna(method='ffill') # forward fill for na values

filtered_data = data_open[data_open['Date'] < date].tail(60)
filtered_array = filtered_data['Open'].to_numpy()

expected_price = data_open[data_open['Date'] == date]
expected_price = expected_price['Open'].iloc[-1]
last_price = filtered_array[-1]
expected_var = expected_price / last_price

scaler = MinMaxScaler(feature_range=(0,1))
scaled_array = scaler.fit_transform(filtered_array.reshape(-1,1))

reshaped_array = scaled_array.reshape(1, 60, 1)

print(reshaped_array)


[[[0.09402967]
  [0.10000018]
  [0.23432854]
  [0.        ]
  [0.05671624]
  [0.10895504]
  [0.15671642]
  [0.2104474 ]
  [0.03880561]
  [0.1313431 ]
  [0.21492573]
  [0.25373134]
  [0.22835803]
  [0.35820896]
  [0.55522406]
  [0.44179069]
  [0.467164  ]
  [0.3686569 ]
  [0.32388078]
  [0.26119403]
  [0.42238788]
  [0.39253713]
  [0.5791043 ]
  [0.46567182]
  [0.55970149]
  [0.38208919]
  [0.27462722]
  [0.39850764]
  [0.42388096]
  [0.0791043 ]
  [0.10447761]
  [0.3029849 ]
  [0.03432818]
  [0.38208919]
  [0.64179104]
  [0.50298525]
  [0.32835821]
  [0.3880597 ]
  [0.10447761]
  [0.3089554 ]
  [0.03432818]
  [0.21791009]
  [0.03432818]
  [0.32835821]
  [0.10746287]
  [0.41343301]
  [0.32388078]
  [0.42985057]
  [0.52238806]
  [0.54328394]
  [0.61641773]
  [0.565672  ]
  [0.65671642]
  [0.68507499]
  [0.86417946]
  [0.94029851]
  [1.        ]
  [0.87462651]
  [0.80597015]
  [0.94029851]]]


In [21]:
# Load the model architecture from model.json
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# Load the model weights into the architecture
loaded_model.load_weights('weights.h5')

# loaded_model.summary()

input_shape = loaded_model.layers[0].input_shape
print("Input shape:", input_shape)

Input shape: (None, 60, 1)


In [22]:
# Now, the model is ready to make predictions on new data
# For instance, if you have new data 'new_data' for prediction:
predicted_prices = loaded_model.predict(reshaped_array)
predicted_prices = scaler.inverse_transform(predicted_prices)
predicted_price = predicted_prices[0][0]
predicted_var = predicted_price / last_price

print(f'last_price: {last_price}')
print(f'expected_price: {expected_price}')
print(f'predicted_price: {predicted_price}')

print(f'expected_var: {expected_var}')
print(f'predicted_var: {predicted_var}')

last_price: 846.0
expected_price: 866.0
predicted_price: 840.9930419921875
expected_var: 1.0236406619385343
predicted_var: 0.9940816099198434
